In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pprint
from datetime import datetime, timedelta
#import tim
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
import impyute as impy
from impyute.imputation.cs import mice
#from sqlalchemy import create_engine
#https://chmodi.tistory.com/114

In [2]:
import psycopg2
from sshtunnel import SSHTunnelForwarder

try:
    tunnel= SSHTunnelForwarder(
         ('210.91.223.248', 35430),
         #ssh_private_key="</path/to/private/ssh/key>",
         ### in my case, I used a password instead of a private key
         ssh_username="medicalai",
         ssh_password="medicalai731!", 
         remote_bind_address=('localhost', 35432))
         
    tunnel.start()
    print("tunnel connected")

    params = {
             'database': 'mimiciv',
             'user': 'mai_admin',
             'password': 'admin1q2w!@',
             'host': 'localhost',
             'port': tunnel.local_bind_port
             }

    conn = psycopg2.connect(**params)
    conn.autocommit = True
    curs = conn.cursor()
    print("database connected")

except Exception as e:
    print("Connection Failed due to {e}")

# Remember to close the SSH tunnel
#tunnel.close()


tunnel connected
database connected


In [10]:
curs.execute("select * from mimiciv_derived.height;")
columns_name = [desc[0] for desc in curs.description]
height = pd.DataFrame(curs.fetchall(), columns=columns_name) 

In [11]:
# height.to_csv('../outputs/height.csv', index=False, encoding='utf-8-sig')

In [12]:
height = pd.read_csv('../outputs/height.csv', parse_dates=['charttime'])

In [14]:
height

,subject_id,stay_id,charttime,height
0,10000032,39553978,2180-07-23 12:36:00,152.0
1,10001725,31205490,2110-04-11 15:52:00,157.0
2,10001884,37510196,2131-01-11 04:20:00,157.0
3,10002013,39060235,2160-05-18 10:00:00,157.0
4,10002428,33987268,2156-04-12 16:24:00,150.0
...,...,...,...,...
33469,19999287,37692584,2197-07-26 03:31:00,163.0
33470,19999287,35165301,2197-08-03 20:58:00,165.0
33471,19999297,37364566,2162-08-16 05:48:00,165.0
33472,19999442,32336619,2148-11-19 14:23:00,193.0


In [15]:
subjectlist1 = pd.read_csv('./data/subjectlist1.csv', parse_dates=['admittime','intubationtime','dischtime', 'deathtime','extubationtime','reintubation_eventtime'])
sub1list = subjectlist1 >> select('subject_id','hadm_id','stay_id')

In [18]:
sub1_height = sub1list >> left_join(height, by=['subject_id','stay_id'])
sub1_height

,subject_id,hadm_id,stay_id,charttime,height
0,10001884,26184834,37510196,2131-01-11 04:20:00,157.0
1,10003400,23559586,38383343,2137-08-10 19:54:00,165.0
2,10004401,27939719,31202136,NaT,NaN
3,10004401,29988601,32773003,2144-01-26 22:28:00,168.0
4,10004401,29988601,32773003,2144-01-26 22:28:00,168.0
...,...,...,...,...,...
8998,19997367,20617667,35616526,2126-04-21 10:08:00,147.0
8999,19997367,20617667,35616526,2126-04-21 10:08:00,147.0
9000,19999068,21606769,30143796,2161-08-24 04:11:00,170.0
9001,19999442,26785317,32336619,2148-11-19 14:23:00,193.0


In [19]:
# sub1_height.to_csv('../outputs/sub1_height.csv', index=False, encoding='utf-8-sig')

In [20]:
sub1_height.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9003 entries, 0 to 9002
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   subject_id  9003 non-null   int64         
 1   hadm_id     9003 non-null   int64         
 2   stay_id     9003 non-null   int64         
 3   charttime   6499 non-null   datetime64[ns]
 4   height      6499 non-null   float64       
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 351.8 KB
